In [1]:
!pip install nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.4 MB/s eta 0:00:00:00:01


In [2]:
import numpy as np
import nibabel as nib
import glob
import os
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [3]:
!dir

Brats_dataloader.ipynb	MNet.ipynb  quick_start_pytorch.ipynb
Data			README.md   quick_start_pytorch_images
Kits_dataloader.ipynb	main.ipynb


In [4]:
TRAIN_DATASET_PATH = "./Data/brats20"
VALIDATION_DATASET_PATH = "./Data/brats20"

In [5]:
test_image_flair=nib.load(TRAIN_DATASET_PATH+"/BraTS20_Training_001/BraTS20_Training_001_flair.nii").get_fdata()
print(test_image_flair.max())
print(test_image_flair.shape)
print(type(test_image_flair))

625.0
(240, 240, 155)
<class 'numpy.memmap'>


In [6]:
test_image_seg=nib.load(TRAIN_DATASET_PATH+"/BraTS20_Training_001/BraTS20_Training_001_seg.nii").get_fdata()
print(test_image_seg.max())
print(test_image_seg.shape)
print(type(test_image_seg))

4.0
(240, 240, 155)
<class 'numpy.memmap'>


In [7]:
test_image_t1=nib.load(TRAIN_DATASET_PATH+"/BraTS20_Training_001/BraTS20_Training_001_t1.nii").get_fdata()
print(test_image_t1.max())
print(test_image_t1.shape)
print(type(test_image_t1))

678.0
(240, 240, 155)
<class 'numpy.memmap'>


In [30]:
test_image_t2=nib.load(TRAIN_DATASET_PATH+"/BraTS20_Training_003/BraTS20_Training_003_t2.nii").get_fdata()
print(test_image_t2.max())
print(test_image_t2.shape)
print(type(test_image_t2))

385.0
(240, 240, 155)
<class 'numpy.memmap'>


In [9]:
test_image_t1ce=nib.load(TRAIN_DATASET_PATH+"/BraTS20_Training_001/BraTS20_Training_001_t1ce.nii").get_fdata()
print(test_image_t1ce.max())
print(test_image_t1ce.shape)
print(type(test_image_t1ce))

1845.0
(240, 240, 155)
<class 'numpy.memmap'>


In [10]:
print(test_image_t1.shape)
print(test_image_t1.shape[-1])
print(test_image_t1.shape[-2])
print(test_image_t1.shape[-3])

(240, 240, 155)
155
240
240


In [11]:
test_image_t1.reshape(-1,test_image_t1.shape[-1]).shape

(57600, 155)

In [12]:
test_image_t1.reshape(-1,test_image_t1.shape[-1]).reshape(test_image_t1.shape).shape

(240, 240, 155)

In [15]:
print(test_image_seg[test_image_seg==4])

[4. 4. 4. ... 4. 4. 4.]


In [6]:
t2_list = sorted(glob.glob(TRAIN_DATASET_PATH+'/*/*t2.nii'))
t1ce_list = sorted(glob.glob(TRAIN_DATASET_PATH+'/*/*t1ce.nii'))
flair_list = sorted(glob.glob(TRAIN_DATASET_PATH+'/*/*flair.nii'))
mask_list = sorted(glob.glob(TRAIN_DATASET_PATH+'/*/*[se][eg][gm].nii')) # handling the 355th file also

In [18]:
print(len(t2_list))
print(t2_list[0]) 

40
./Data/brats20/BraTS20_Training_001/BraTS20_Training_001_t2.nii


In [18]:
from torch.utils.data import Dataset

class BratsDataset(Dataset):
    def __init__(self,t2s,t1s,flairs,masks):
        self.t2 = t2s
        self.t1 = t1s
        self.flair = flairs
        self.mask = masks
    
    def __len__(self):
        return len(self.t2)
    
    def __getitem__(self,index):
        return self.t2[index],self.t1[index],self.flair[index],self.mask[index]
    
t2_img_list = []
t1_img_list = []
flair_img_list = []
mask_img_list = []

for i in range(10,15):#range(len(t2_list)):
    print(t2_list[i])
    temp_t2 = nib.load(t2_list[i]).get_fdata()
    temp_t2 = scaler.fit_transform(temp_t2.reshape(-1,temp_t2.shape[-1])).reshape(temp_t2.shape)
    
    temp_t1=nib.load(t1ce_list[i]).get_fdata()
    temp_t1=scaler.fit_transform(temp_t1.reshape(-1, temp_t1.shape[-1])).reshape(temp_t1.shape)
   
    temp_flair=nib.load(flair_list[i]).get_fdata()
    temp_flair=scaler.fit_transform(temp_flair.reshape(-1, temp_flair.shape[-1])).reshape(temp_flair.shape)
        
    temp_mask=nib.load(mask_list[i]).get_fdata()
    temp_mask=temp_mask.astype(np.uint8)
    temp_mask[temp_mask==4] = 3 
    
    t2_img_list.append(temp_t2)
    t1_img_list.append(temp_t1)
    flair_img_list.append(temp_flair)
    mask_img_list.append(temp_mask)
    
brats_dataset = BratsDataset(t2_img_list,t1_img_list,flair_img_list,mask_img_list)

./Data/brats20/BraTS20_Training_011/BraTS20_Training_011_t2.nii
./Data/brats20/BraTS20_Training_012/BraTS20_Training_012_t2.nii
./Data/brats20/BraTS20_Training_013/BraTS20_Training_013_t2.nii
./Data/brats20/BraTS20_Training_014/BraTS20_Training_014_t2.nii
./Data/brats20/BraTS20_Training_015/BraTS20_Training_015_t2.nii
